## Imports and Functions

In [ ]:
from igorwriter import IgorWave
from typing import List, Optional, Union
import re

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
# plt.style.use('ggplot')
import plotly.graph_objects as go

from scipy.ndimage import gaussian_filter

import os
import pandas as pd

import sys
sys.path.append(r'C:\Users\atully\Code\GitHub\ARPES Code\arpes-code-python')
from arpes_functions import analysis_functions, plotting_functions, HDF5_loader, misc_functions, kw_data_loader, fitting_functions
from arpes_functions import filter_functions as ff

In [ ]:
# in cnn.py

def save_to_igor_itx(file_path: str, xs: List[np.ndarray], datas: List[np.ndarray], names: List[str],
                     ys: Optional[List[np.ndarray]] = None,
                     x_labels: Optional[Union[str, List[str]]] = None,
                     y_labels: Optional[Union[str, List[str]]] = None):
    """Save data to a .itx file which can be dropped into Igor"""

    def check_axis_linear(arr: np.ndarray, axis: str, name: str, current_waves: list) -> bool:
        if arr.shape[-1] > 1 and not np.all(np.isclose(np.diff(arr), np.diff(arr)[0])):
            logger.warning(f"{file_path}: Igor doesn't support a non-linear {axis}-axis. Saving as separate wave")
            axis_wave = IgorWave(arr, name=name + f'_{axis}')
            current_waves.append(axis_wave)
            return False
        else:
            return True

    if x_labels is None or isinstance(x_labels, str):
        x_labels = [x_labels] * len(datas)
    if y_labels is None or isinstance(y_labels, str):
        y_labels = [y_labels] * len(datas)
    if ys is None:
        ys = [None] * len(datas)
    assert all([len(datas) == len(list_) for list_ in [xs, names, x_labels, y_labels]])

    waves = []
    for x, y, data, name, x_label, y_label in zip(xs, ys, datas, names, x_labels, y_labels):
        wave = IgorWave(data, name=name)
        if x is not None:
            if check_axis_linear(x, 'x', name, waves):
                wave.set_dimscale('x', x[0], np.mean(np.diff(x)), units=x_label)
        if y is not None:
            if check_axis_linear(y, 'y', name, waves):
                wave.set_dimscale('y', y[0], np.mean(np.diff(y)), units=y_label)
        elif y_label is not None:
            wave.set_datascale(y_label)
        waves.append(wave)

    with open(file_path, 'w') as fp:
        for wave in waves:
            wave.save_itx(fp, image=True)  # Image = True hopefully makes np and igor match in x/y

In [ ]:
# in cnn.py

def fix_itx_format(filename: str = 'test.itx'):
    with open(filename, 'r') as f:
        lines = f.readlines()
    #     for i in range(4):
    #         lines.append(f.readline())

    for ln in [1, -3, -2, -1]:
        lines[ln] = lines[ln].replace("'", "")


    new = lines[:-6]
    new[1] = 'WAVES/D/N='+lines[1][12:]
    new.append('END\n')

    dscale = lines[-3][2:-1].replace(',', ' ', 1)
    xscale = lines[-2][2:-1].replace(',', ' ', 1).replace(' ', '', 1)
    yscale = lines[-1][2:-1].replace(',', ' ', 1).replace(' ', '', 1)

    new.append(f'X {xscale}; {yscale}; {dscale}'.replace('"",', '"", '))

    with open('test_fixed.itx', 'w') as f:
        f.writelines(new)

In [ ]:
# in loading_functions.py

def load_denoised_data(file_path: str, filename: str):
    data_path = os.path.join(file_path, filename)
    df = pd.read_csv(data_path, skiprows=3, header=None, sep='\s+', skipfooter=2)
    data = np.array(df).T
    with open(data_path, 'r') as f:
        lines = f.readlines()
    last = lines[-1]

    x_start, x_step = [float(v) for v in re.search('x\s*(-?\d*.\d+),\s*(-?\d+.\d+)', last).groups()]
    y_start, y_step = [float(v) for v in re.search('y\s*(-?\d*.\d+),\s*(-?\d+.\d+)', last).groups()]

    x = np.linspace(x_start, x_start+data.shape[1]*x_step, data.shape[1])
    y = np.linspace(y_start, y_start+data.shape[0]*y_step, data.shape[0])
    return x, y, data    

In [ ]:
# in fitting_functions.py

def fit_partial_cone(x, y, data, xlim, ylim, centers=[0.25, 0.4], num_peaks=2, window=0.02, steps=20, offset_type='quadratic', plot=True):
    
    x_fit, y_fit, d_fit = analysis_functions.limit_dataset(x, y, data, xlim=(xlim[0], xlim[1]), ylim=(ylim[0], ylim[1]))

    # fit top of cone
    if num_peaks == 1:
        fits = []
        coords = []

        for yval in np.linspace(ylim[0], ylim[1], steps):
            row = analysis_functions.get_averaged_slice(analysis_functions.get_horizontal_slice(d_fit, y_fit, yval, window), axis='y')

            fit = fitting_functions.fit_lorentzian_data(x=x_fit, data=row,
                                         num_peaks=num_peaks, amplitudes=0.5, centers=[centers[0]], sigmas=0.05,
                                         offset_type=offset_type)
            fits.append(fit)
            coords.extend([(fit.best_values[f'i{i}_center'], yval) for i in range(num_peaks)])
        return coords
        
    if num_peaks == 2:
        fits1 = []
        coords1 = []

        for yval in np.linspace(ylim[0], ylim[1], steps):

            row = analysis_functions.get_averaged_slice(analysis_functions.get_horizontal_slice(d_fit, y_fit, yval, window), axis='y')
            fit1 = fitting_functions.fit_lorentzian_data(x=x_fit, data=row,
                                         num_peaks=num_peaks, amplitudes=0.5, centers=[centers[0], centers[1]], sigmas=0.05,
                                         offset_type=offset_type)
            fits1.append(fit1)
            coords1.extend([(fit1.best_values[f'i{i}_center'], yval) for i in range(num_peaks)])
        return coords1
    
    else:
        return ValueError(f'num_peaks is {num_peaks}; must be 1 or 2')

In [ ]:
# in fitting_functions.py

def compile_peaks(coords_top, coords_middle, coords_bottom):
    x_line1, y_line1 = np.array([c[0] for c in coords_top]), np.array([c[1] for c in coords_top])
    
    x_line2 = np.append(x_line1, np.array([c[0] for c in coords_middle]), axis=0)
    x_line_all = np.append(x_line2, np.array([c[0] for c in coords_bottom]), axis=0)

    y_line2 = np.append(y_line1, np.array([c[1] for c in coords_middle]), axis=0)
    y_line_all = np.append(y_line2, np.array([c[1] for c in coords_bottom]), axis=0)
    
    return x_line_all, y_line_all

In [ ]:
# in analysis_functions.py

def fit_cone_lines(x_line, y_line, cone_center=None):
    
    if cone_center is None:
        cone_center = np.mean(x_line)
    # Fits all data points (RIGHT SIDE OF CONE)
    fit_line1 = fitting_functions.fit_linear_data(x=x_line[np.where(x_line > cone_center)],
                              data=y_line[np.where(x_line > cone_center)],
                              num=1,
                              aes=1, bes=1,
                              offset_type=None)

    # Fits all data points (LEFT SIDE OF CONE)
    fit_line2 = fitting_functions.fit_linear_data(x=x_line[np.where(x_line < cone_center)],
                              data=y_line[np.where((x_line < cone_center))],
                              num=1,
                              aes=1, bes=1,
                              offset_type=None)
   
    return fit_line1, fit_line2

In [ ]:
# in misc_functions.py

def limit_coords(x_coords, y_coords, xlim, ylim=None):
    x_subset=x_line[np.where(np.logical_and(x_coords > xlim[0], x_coords < xlim[1]))]
    y_subset=y_line[np.where(np.logical_and(x_coords > xlim[0], x_coords < xlim[1]))]
    if ylim:
        x_subset=x_subset[np.where(np.logical_and(y_coords > ylim[0], y_coords < ylim[1]))]
        y_subset=y_subset[np.where(np.logical_and(y_coords > ylim[0], y_coords < ylim[1]))]
    return x_subset, y_subset

# XUV Diffraction Data

In [ ]:
fp= r'E:\atully\k-corrected data\Apr_2021\XUV_FS_gamma0'
fn = r'XUV_FS_gamma0_gkw11_filteredFFT_0.00int.h5'
data, theta, phi, energy = HDF5_loader.load_hdf5(fp, fn)  # load data from hdf5

In [ ]:
# Get Raw Dataslice

# set slice parameters
slice_dim='y'
int_range=0.2
slice_val=18.3
# slice_val=18.1
# slice_val=17.9

# analysis -- get slice, FFT, limit dataset, normalize dataset
xaxis, yaxis, dataslice = analysis_functions.get_2Dslice(x=theta, y=energy, z=phi, data=data, slice_dim=slice_dim, slice_val=slice_val, int_range=int_range)
x, y, d = analysis_functions.limit_dataset(x=xaxis, y=yaxis, data=dataslice, xlim=(-0.73, 0.52), ylim=(-1.4, 0.1))
d = analysis_functions.norm_data(d)

# plot slice
fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=d, xlabel='kx', ylabel='ky', title=f'Ek={slice_val}', cmap='Blues')

plotting_functions.format_FS_plot(fig, ax, slice_val=slice_val, EF=18.3)

# plt.savefig(r'C:\Users\XXX.png')

In [ ]:
# Denoised Data
fp_dn = r'C:\Users\atully\OneDrive\Physics.UBC\DAE\XUV diffraction data\FS'
fn_dn = 'XUV_FS_EF_dn.itx'

title = re.search('FS_(.*?)\_', fn_dn).group()
title = title[3:-1]

x, y, data_dn = load_denoised_data(fp_dn, fn_dn)

d = analysis_functions.norm_data(data_dn)

# plot slice
fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=d, xlabel='kx', ylabel='ky', cmap='Blues')

plotting_functions.format_FS_plot(fig, ax, EF=18.3, title=title)

# Fitting XUV ky=-0.36 Cones

## Fitting Denoised ky=-0.36 Data

In [ ]:
# Denoised Data
fp_dn = r'C:\Users\atully\OneDrive\Physics.UBC\DAE\XUV diffraction data\xuv_ky=-0.36'
fn_dn = 'xuv_ky=-0.36_dn.itx'

title = 'ky=-0.36'

x_dn, y_dn, data_dn = load_denoised_data(fp_dn, fn_dn)

d_dn = analysis_functions.norm_data(data_dn)

# plot slice
fig, ax = plotting_functions.plot_2D_mpl(x=x_dn, y=y_dn, data=d_dn, xlabel='kx', ylabel='ky', cmap='Blues', EF=18.3)

# plotting_functions.format_FS_plot(fig, ax, EF=18.3, title=title)
plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=18.3, slice_constant='ky', title=title)

In [ ]:
xfit, yfit, datafit = x_dn, y_dn, data_dn

coords_top = fit_partial_cone(xfit, yfit, datafit, xlim=(-0.4, 0.0), ylim=(18.1, 18.2), centers=[-0.2], num_peaks=1, window=0.05)  # top of left cone
coords_bottom = fit_partial_cone(xfit, yfit, datafit, xlim=(-0.38, 0.0), ylim=(17.6, 17.7), centers=[-0.25, -0.1], num_peaks=2, window=0.02)  # bottom of left  cone
coords_middle = fit_partial_cone(xfit, yfit, datafit, xlim=(-0.3, -0.07), ylim=(17.7, 17.8), centers=[-0.25, -0.1], num_peaks=2, window=0.02)  # middle of left  cone

x_line, y_line = compile_peaks(coords_top, coords_middle, coords_bottom)  # compile coords of all peaks 
fit1, fit2 = fit_cone_lines(x_line, y_line)  # linear fits

In [ ]:
from IPython.display import display

display(fit1)

In [ ]:
EF = 18.3

fig, ax = plotting_functions.plot_2D_mpl(x=x_dn, y=y_dn, data=d_dn, xlabel='kx', ylabel='ky', cmap='Blues', EF=EF)
ax.scatter(x=[c[0] for c in coords_bottom], y=[c[1]-EF for c in coords_bottom], color='xkcd:hot pink')
ax.scatter(x=[c[0] for c in coords_middle], y=[c[1]-EF for c in coords_middle], color='xkcd:dark green')
ax.scatter(x=[c[0] for c in coords_top], y=[c[1]-EF for c in coords_top], color='xkcd:bright purple')

plt.plot(x, fit1.eval(x=x)-EF, color='xkcd:turquoise', linestyle='--', linewidth=3)
plt.plot(x, fit2.eval(x=x)-EF, color='xkcd:turquoise', linestyle='--', linewidth=3)

ax.set_xlim(-0.38, 0.0)
ax.set_ylim(-0.75, 0.1)
plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=EF, slice_constant='ky')

In [ ]:
# Get Raw Dataslice

# set slice parameters
slice_dim='z'
int_range=0.1
slice_val=-0.36

# analysis -- get slice of FFT_filtered dataset, limit dataset, normalize dataset
xaxis, yaxis, dataslice = analysis_functions.get_2Dslice(x=theta, y=energy, z=phi, data=data, slice_dim=slice_dim, slice_val=slice_val, int_range=int_range)
x, y, d = analysis_functions.limit_dataset(xaxis, yaxis, dataslice, xlim=(-0.5, 0.5), ylim=(17.55, 18.4))
normed_data = analysis_functions.norm_data(d)

# plot slice
fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=normed_data, xlabel='kx', ylabel='E - EF (eV)', title=f'ky={slice_val}', cmap='Blues', EF=18.3)

plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=18.3, slice_constant='ky')

plt.plot(x, fit1.eval(x=x)-EF, color='xkcd:turquoise', linestyle='--', linewidth=3)
plt.plot(x, fit2.eval(x=x)-EF, color='xkcd:turquoise', linestyle='--', linewidth=3)

ax.set_xlim(-0.38, 0.0)
ax.set_ylim(-0.75, 0.1)
plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=18.3, slice_constant='ky')

## Fitting Raw ky=-0.36 Data

In [ ]:
# Get Raw Dataslice

# set slice parameters
slice_dim='z'
int_range=0.1
slice_val=-0.36

# analysis -- get slice of FFT_filtered dataset, limit dataset, normalize dataset
xaxis, yaxis, dataslice = analysis_functions.get_2Dslice(x=theta, y=energy, z=phi, data=data, slice_dim=slice_dim, slice_val=slice_val, int_range=int_range)
x, y, d = analysis_functions.limit_dataset(xaxis, yaxis, dataslice, xlim=(-0.5, 0.5), ylim=(17.55, 18.4))
# x, y, d = limit_dataset(xaxis, yaxis, dataslice, xlim=(-0.4, 0.4), ylim=(17.55, 18.4))
normed_data = analysis_functions.norm_data(d)

# plot slice
# fig, ax = plotting_functions.plot_2D_mpl(x=xaxis, y=yaxis, data=dataslice, xlabel='kx', ylabel='E - EF (eV)', title=f'ky={slice_val}', cmap='Blues', EF=18.3)
fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=normed_data, xlabel='kx', ylabel='E - EF (eV)', title=f'ky={slice_val}', cmap='Blues', EF=18.3)

plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=18.3, slice_constant='ky')

In [ ]:
xfit, yfit, datafit = x, y, normed_data

# Same Fit Parameters As Above (for DN Data)
coords_top = fit_partial_cone(xfit, yfit, datafit, xlim=(-0.4, 0.0), ylim=(18.1, 18.2), centers=[-0.2], num_peaks=1, window=0.05)  # top of left cone
coords_bottom = fit_partial_cone(xfit, yfit, datafit, xlim=(-0.38, 0.0), ylim=(17.6, 17.7), centers=[-0.25, -0.1], num_peaks=2, window=0.02)  # bottom of left  cone
coords_middle = fit_partial_cone(xfit, yfit, datafit, xlim=(-0.3, -0.07), ylim=(17.7, 17.8), centers=[-0.25, -0.1], num_peaks=2, window=0.02)  # middle of left  cone

x_line, y_line = compile_peaks(coords_top, coords_middle, coords_bottom)  # compile coords of all peaks 
fit3, fit4 = fit_cone_lines(x_line, y_line, cone_center=-0.185)  # linear fits
# fit3, fit4 = fit_cone_lines(x_line, y_line)  # linear fits

In [ ]:
np.mean(x_line)

In [ ]:
EF = 18.3

fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=normed_data, xlabel='kx', ylabel='ky', cmap='Blues', EF=EF)
ax.scatter(x=[c[0] for c in coords_bottom], y=[c[1]-EF for c in coords_bottom], color='xkcd:hot pink')
ax.scatter(x=[c[0] for c in coords_middle], y=[c[1]-EF for c in coords_middle], color='xkcd:dark green')
ax.scatter(x=[c[0] for c in coords_top], y=[c[1]-EF for c in coords_top], color='xkcd:bright purple')

ax.axvline(x=np.mean(x_line), color='xkcd:bright blue', linestyle='--')  # shows mean of x coords

plt.plot(x, fit3.eval(x=x)-EF, color='xkcd:purple', linestyle='--', linewidth=3)
plt.plot(x, fit4.eval(x=x)-EF, color='xkcd:purple', linestyle='--', linewidth=3)

ax.set_xlim(-0.38, 0.0)
ax.set_ylim(-0.75, 0.1)
plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=EF, slice_constant='ky')

In [ ]:
# Denoised Data with Raw Data Fit on Top
EF = 18.3

fig, ax = plotting_functions.plot_2D_mpl(x=x_dn, y=y_dn, data=d_dn, xlabel='kx', ylabel='ky', cmap='Blues', EF=EF)

plt.plot(x, fit3.eval(x=x)-EF, color='xkcd:purple', linestyle='--', linewidth=3)
plt.plot(x, fit4.eval(x=x)-EF, color='xkcd:purple', linestyle='--', linewidth=3)

ax.set_xlim(-0.38, 0.0)
ax.set_ylim(-0.75, 0.1)
plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=EF, slice_constant='ky')

In [ ]:
# More Stringent Fitting Requirements

# Fits all data points (RIGHT SIDE OF CONE)
fit_line3 = fitting_functions.fit_linear_data(x=x_line[np.where(np.logical_and(x_line < 0, x_line > -0.2))],
                          data=y_line[np.where((np.logical_and(x_line < 0, x_line > -0.2)))],
                          num=1,
                          aes=1, bes=1,
                          offset_type=None)

# Fits all data points (LEFT SIDE OF CONE)
fit_line4 = fitting_functions.fit_linear_data(x=x_line[np.where(np.logical_and(x_line < -0.171, x_line > -0.3))],
                          data=y_line[np.where(np.logical_and(x_line < -0.171, x_line > -0.3))],
                          num=1,
                          aes=1, bes=1,
                          offset_type=None)

In [ ]:
# Raw Data with Strict Fit

fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=normed_data, xlabel='kx', ylabel='ky', cmap='Blues', EF=18.3)

plt.plot(x, fit_line3.eval(x=x)-EF, color='xkcd:purple', linestyle='--', linewidth=3)
plt.plot(x, fit_line4.eval(x=x)-EF, color='xkcd:purple', linestyle='--', linewidth=3)

ax.set_xlim(-0.38, 0.0)
ax.set_ylim(-0.75, 0.1)
plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=18.3, slice_constant='ky')

In [ ]:
# Denoised Data with Raw Data Fit on Top

fig, ax = plotting_functions.plot_2D_mpl(x=x_dn, y=y_dn, data=d_dn, xlabel='kx', ylabel='ky', cmap='Blues', EF=18.3)

plt.plot(x, fit_line3.eval(x=x)-EF, color='xkcd:purple', linestyle='--', linewidth=3)
plt.plot(x, fit_line4.eval(x=x)-EF, color='xkcd:purple', linestyle='--', linewidth=3)

ax.set_xlim(-0.38, 0.0)
ax.set_ylim(-0.75, 0.1)
plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=18.3, slice_constant='ky')

## Explain Fitting Procedure

In [ ]:
# set slice parameters
slice_dim='z'
int_range=0.1
slice_val=-0.36

# analysis -- get slice of FFT_filtered dataset, limit dataset, normalize dataset
xaxis, yaxis, dataslice = analysis_functions.get_2Dslice(x=theta, y=energy, z=phi, data=data, slice_dim=slice_dim, slice_val=slice_val, int_range=int_range)
x, y, d = analysis_functions.limit_dataset(xaxis, yaxis, dataslice, xlim=(-0.5, 0.5), ylim=(17.55, 18.4))
normed_data = analysis_functions.norm_data(d)

fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=normed_data, xlabel='kx', ylabel='E - EF (eV)', title=f'ky={slice_val}', cmap='Blues', EF=18.3)

ax.set_xlim(-0.4, 0.0)
plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=18.3, slice_constant='ky')

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\Figs\fit_procedure_exp1.png')

In [ ]:
xfit, yfit, datafit = x, y, normed_data
window = 0.02
coords_top = fit_partial_cone(xfit, yfit, datafit, xlim=(-0.4, 0.0), ylim=(18.1, 18.2), centers=[-0.2], num_peaks=1, window=window)  # top of left cone

In [ ]:
EF = 18.3
yval = 18.15

fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=normed_data, xlabel='kx', ylabel='ky', cmap='Blues', EF=EF)

ax.scatter(x=[c[0] for c in coords_top], y=[c[1]-EF for c in coords_top], color='xkcd:bright purple')

ax.axhline(y=yval-EF, xmin=-1, xmax=1, linestyle='dashed', color='red', linewidth=2)
ax.axhline(y=yval-EF+window, xmin=-1, xmax=1, linestyle='dotted', color='xkcd:bright green', linewidth=2)
ax.axhline(y=yval-EF-window, xmin=-1, xmax=1, linestyle='dotted', color='xkcd:bright green', linewidth=2)

plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=EF, slice_constant='ky')
ax.set_xlim(-0.4, 0.0)
ax.set_ylim(-0.3, 0.1)

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\Figs\fit_procedure_exp2.png')

In [ ]:
x_fit, y_fit, d_fit = analysis_functions.limit_dataset(x, y, normed_data, xlim=(-0.4, 0.0), ylim=(18.1, 18.2))

row = analysis_functions.get_averaged_slice(analysis_functions.get_horizontal_slice(d_fit, y_fit, yval, window), axis='y')
plt.scatter(x=x_fit, y=row)

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\Figs\fit_procedure_exp3.png')

In [ ]:
fit = fitting_functions.fit_lorentzian_data(x=x_fit, data=row,
                             num_peaks=1, amplitudes=0.5, centers=[-0.02], sigmas=0.5,
                             offset_type='quadratic')
# print(fit.fit_report())

fig, ax = plt.subplots(1)
ax.plot(x_fit, row, label='data for fit')
ax.scatter(x=x_fit, y=fit.eval(x=x_fit))

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\Figs\fit_procedure_exp4.png')

In [ ]:
peak = (fit.best_values[f'i0_center'], yval)
peak

In [ ]:
fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=normed_data, xlabel='kx', ylabel='ky', cmap='Blues', EF=EF)

ax.scatter(x=[c[0] for c in coords_top], y=[c[1]-EF for c in coords_top], color='xkcd:bright purple')

ax.axhline(y=yval-EF, xmin=-1, xmax=1, linestyle='dashed', color='red', linewidth=2)
ax.axhline(y=yval-EF+window, xmin=-1, xmax=1, linestyle='dotted', color='xkcd:bright green', linewidth=2)
ax.axhline(y=yval-EF-window, xmin=-1, xmax=1, linestyle='dotted', color='xkcd:bright green', linewidth=2)

ax.scatter(x=peak[0], y=peak[1]-EF, color='xkcd:bright blue')

plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=EF, slice_constant='ky')
ax.set_xlim(-0.4, 0.0)
ax.set_ylim(-0.3, 0.1)

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\Figs\fit_procedure_exp5.png')

# Fitting XUV ky=-0.72 Cones

In [ ]:
# set slice parameters
slice_dim='z'
int_range=0.1
slice_val=-0.72

# analysis -- get slice of FFT_filtered dataset, limit dataset, normalize dataset
xaxis, yaxis, dataslice = analysis_functions.get_2Dslice(x=theta, y=energy, z=phi, data=data, slice_dim=slice_dim, slice_val=slice_val, int_range=int_range)
x, y, d = analysis_functions.limit_dataset(xaxis, yaxis, dataslice, xlim=(-0.5, 0.5), ylim=(17.55, 18.4))
normed_data = analysis_functions.norm_data(d)

# plot slice
# fig, ax = plotting_functions.plot_2D_mpl(x=xaxis, y=yaxis, data=dataslice, xlabel='kx', ylabel='E - EF (eV)', title=f'ky={slice_val}', cmap='Blues', EF=18.3)
fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=normed_data, xlabel='kx', ylabel='E - EF (eV)', title=f'ky={slice_val}', cmap='Blues', EF=18.3)

plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=18.3, slice_constant='ky')

In [ ]:
# Denoised Data
fp_dn = r'C:\Users\atully\OneDrive\Physics.UBC\DAE\XUV diffraction data\xuv_ky=-0.72'
fn_dn = 'xuv_ky=-0.72_dn.itx'

title = 'ky=-0.72'

x_dn, y_dn, data_dn = load_denoised_data(fp_dn, fn_dn)

d_dn = analysis_functions.norm_data(data_dn)

# plot slice
fig, ax = plotting_functions.plot_2D_mpl(x=x_dn, y=y_dn, data=d_dn, xlabel='kx', ylabel='ky', cmap='Blues', EF=18.3)

plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=18.3, slice_constant='ky', title=title)

# Lamp Diffraction Data

In [ ]:
fp = r'E:\atully\k-corrected data\Apr_2021\phi_motor_scan_FermiEdge2'
fn = r'phi_FermiEdge2kw_filteredFFT_0.00int.h5'
data, theta, phi, energy = HDF5_loader.load_hdf5(fp, fn)  # load data from hdf5

In [ ]:
# set slice parameters
slice_dim='y'
int_range=0.2
slice_val=16.8

# analysis -- get slice, FFT, limit dataset, normalize dataset
xaxis, yaxis, dataslice = analysis_functions.get_2Dslice(x=theta, y=energy, z=phi, data=data, slice_dim=slice_dim, slice_val=slice_val, int_range=int_range)
x, y, d = analysis_functions.limit_dataset(xaxis, yaxis, dataslice, xlim=(-0.65, 0.52), ylim=(-1.4, 0.1))
normed_data = analysis_functions.norm_data(d)

# plot slice
fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=normed_data, xlabel='kx', ylabel='ky', title=f'Ek={slice_val}', cmap='Blues')

plotting_functions.format_FS_plot(fig, ax, slice_val=slice_val, EF=16.8)

In [ ]:
# Denoised Data
fp_dn = r'C:\Users\atully\OneDrive\Physics.UBC\DAE\Lamp diffraction data\FS'
fn_dn = 'Ek=16.8_dn.itx'

title='EF'

x, y, data_dn = load_denoised_data(fp_dn, fn_dn)

d = analysis_functions.norm_data(data_dn)

# plot slice
fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=d, xlabel='kx', ylabel='ky', cmap='Blues')

plotting_functions.format_FS_plot(fig, ax, EF=16.8, title=title)

# Fitting Lamp ky=-0.72 Cones

## Fitting Raw ky=-0.72 Lamp Data

In [ ]:
# NOTE INT RANGE!

# set slice parameters
EF = 16.8
slice_dim='z'
int_range=0.05
# slice_val=-0.71
slice_val=-0.72

# analysis -- get slice of FFT_filtered dataset, limit dataset, normalize dataset
xaxis, yaxis, dataslice = analysis_functions.get_2Dslice(x=theta, y=energy, z=phi, data=data, slice_dim=slice_dim, slice_val=slice_val, int_range=int_range)
# x, y, d = limit_dataset(xaxis, yaxis, dataslice, xlim=(-0.5, 0.5), ylim=(15.6, 16.8))
x, y, d = analysis_functions.limit_dataset(xaxis, yaxis, dataslice, xlim=(-0.5, 0.5), ylim=(16.0, 16.8))
normed_data = analysis_functions.norm_data(d)

# plot slice
fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=normed_data, xlabel='kx', ylabel='E - EF (eV)', title=f'ky={slice_val}', cmap='Blues', EF=EF)

plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=18.3, slice_constant='ky')

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\PhD\Dissertation\Data\Fitted Data\lamp_ky=-0.72.png')  # int range 0.05 
# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\Figs\lamp_raw_ky=-0.72.png')  # int range 0.05 

In [ ]:
# Fit Data

xfit, yfit, datafit = x, y, normed_data

coords_top = fit_partial_cone(xfit, yfit, datafit, xlim=(0.0, 0.5), ylim=(16.6, 16.7), centers=[0.35], num_peaks=1)  # top of right cone
coords_bottom = fit_partial_cone(xfit, yfit, datafit, xlim=(0.25, 0.45), ylim=(16.1, 16.3), centers=[0.25, 0.4], num_peaks=2)  # bottom of right  cone
coords_middle = fit_partial_cone(xfit, yfit, datafit, xlim=(0.25, 0.45), ylim=(16.3, 16.5), centers=[0.3, 0.4], num_peaks=2)  # middle of right  cone

x_line, y_line = compile_peaks(coords_top, coords_middle, coords_bottom)  # compile coords of all peaks 
fit1, fit2 = fit_cone_lines(x_line, y_line)  # linear fits

In [ ]:
# Plot Data with Fit

EF = 16.8

fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=normed_data, xlabel='kx', ylabel='ky', cmap='Blues', EF=EF)
ax.scatter(x=[c[0] for c in coords_bottom], y=[c[1]-EF for c in coords_bottom], color='xkcd:hot pink')
ax.scatter(x=[c[0] for c in coords_middle], y=[c[1]-EF for c in coords_middle], color='xkcd:dark green')
ax.scatter(x=[c[0] for c in coords_top], y=[c[1]-EF for c in coords_top], color='xkcd:bright purple')

# ax.axvline(x=np.mean(x_line), color='xkcd:bright blue', linestyle='--')  # shows mean of x coords

plt.plot(x, fit1.eval(x=x)-EF, color='xkcd:purple', linestyle='--', linewidth=3)
plt.plot(x, fit2.eval(x=x)-EF, color='xkcd:purple', linestyle='--', linewidth=3)

ax.set_xlim(0.0, 0.5)
ax.set_ylim(-0.75, 0.0)
plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=EF, slice_constant='ky')

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\Figs\lamp_raw_ky=-0.72_fit1.png')  # int range 0.05 

In [ ]:
x_subset, y_subset = limit_coords(x_line, y_line, xlim=[0.28, 0.44])

# Plot Data
EF = 16.8

fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=normed_data, xlabel='kx', ylabel='ky', cmap='Blues', EF=EF)

# ax.axvline(x=np.mean(x_line), color='xkcd:bright blue', linestyle='--')  # shows mean of x coords
ax.scatter(x=x_line, y=y_line-EF, color='xkcd:bright blue')
ax.scatter(x=x_subset, y=y_subset-EF, color='xkcd:bright pink')

ax.set_xlim(0.0, 0.5)
ax.set_ylim(-0.75, 0.0)
plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=EF, slice_constant='ky')

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\Figs\lamp_raw_ky=-0.72_fit2.png')  # int range 0.05 

In [ ]:
# Fit Data
fit3, fit4 = fit_cone_lines(x_subset, y_subset)  # linear fits

In [ ]:
# Plot Data with Fit

EF = 16.8

fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=normed_data, xlabel='kx', ylabel='ky', cmap='Blues', EF=EF)

ax.scatter(x=x_subset, y=y_subset-EF, color='xkcd:bright pink')

# ax.axvline(x=np.mean(x_line), color='xkcd:bright blue', linestyle='--')  # shows mean of x coords

plt.plot(x, fit3.eval(x=x)-EF, color='xkcd:purple', linestyle='--', linewidth=3)
plt.plot(x, fit4.eval(x=x)-EF, color='xkcd:purple', linestyle='--', linewidth=3)

ax.set_xlim(0.0, 0.5)
ax.set_ylim(-0.75, 0.0)
plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=EF, slice_constant='ky')

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\Figs\lamp_raw_ky=-0.72_fit3.png')  # int range 0.05 

## Fitting Denoised ky=-0.72 Lamp Data

In [ ]:
# Denoised Data
fp_dn = r'C:\Users\atully\OneDrive\Physics.UBC\DAE\Lamp diffraction data\ky=-0.72'
fn_dn = 'ky=-0.72_dn.itx'

title = 'ky=-0.72'

x_dn, y_dn, data_dn = load_denoised_data(fp_dn, fn_dn)

d_dn = analysis_functions.norm_data(data_dn)

# plot slice
fig, ax = plotting_functions.plot_2D_mpl(x=x_dn, y=y_dn, data=d_dn, xlabel='kx', ylabel='ky', cmap='Blues', EF=16.8)

# plotting_functions.format_FS_plot(fig, ax, EF=18.3, title=title)
plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=16.8, slice_constant='ky', title=title)

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\PhD\Dissertation\Data\Fitted Data\lamp_ky=-0.72_dn.png')
# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\Figs\lamp_dn_ky=-0.72.png')

In [ ]:
xfit, yfit, datafit = x_dn, y_dn, d_dn

coords_top = fit_partial_cone(xfit, yfit, datafit, xlim=(0.0, 0.5), ylim=(16.6, 16.7), centers=[0.35], num_peaks=1)  # top of right cone
coords_bottom = fit_partial_cone(xfit, yfit, datafit, xlim=(0.25, 0.45), ylim=(16.1, 16.3), centers=[0.25, 0.4], num_peaks=2)  # bottom of right  cone
coords_middle = fit_partial_cone(xfit, yfit, datafit, xlim=(0.25, 0.45), ylim=(16.3, 16.5), centers=[0.3, 0.4], num_peaks=2)  # middle of right  cone

x_line, y_line = compile_peaks(coords_top, coords_middle, coords_bottom)  # compile coords of all peaks 
fit1_dn, fit2_dn = fit_cone_lines(x_line, y_line)  # linear fits

In [ ]:
fig, ax = plotting_functions.plot_2D_mpl(x=x_dn, y=y_dn, data=d_dn, xlabel='kx', ylabel='ky', cmap='Blues', EF=16.8)
ax.scatter(x=[c[0] for c in coords_bottom], y=[c[1]-EF for c in coords_bottom], color='xkcd:hot pink')
ax.scatter(x=[c[0] for c in coords_middle], y=[c[1]-EF for c in coords_middle], color='xkcd:dark green')
ax.scatter(x=[c[0] for c in coords_top], y=[c[1]-EF for c in coords_top], color='xkcd:bright purple')

plt.plot(x, fit1_dn.eval(x=x)-EF, color='xkcd:turquoise', linestyle='--', linewidth=3)
plt.plot(x, fit2_dn.eval(x=x)-EF, color='xkcd:turquoise', linestyle='--', linewidth=3)

ax.set_xlim(0.0, 0.5)
ax.set_ylim(-0.75, 0.0)
plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=16.8, slice_constant='ky')

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\Figs\lamp_dn_ky=-0.72_fit1.png')  # int range 0.05 

In [ ]:
x_subset, y_subset = limit_coords(x_line, y_line, xlim=[0.28, 0.44])

# Fit Data
fit3_dn, fit4_dn = fit_cone_lines(x_subset, y_subset)  # linear fits

In [ ]:
# Plot Data with Fit

EF = 16.8

fig, ax = plotting_functions.plot_2D_mpl(x=x_dn, y=y_dn, data=d_dn, xlabel='kx', ylabel='ky', cmap='Blues', EF=EF)

ax.scatter(x=x_subset, y=y_subset-EF, color='xkcd:bright pink')

# ax.axvline(x=np.mean(x_line), color='xkcd:bright blue', linestyle='--')  # shows mean of x coords

plt.plot(x, fit3_dn.eval(x=x)-EF, color='xkcd:turquoise', linestyle='--', linewidth=3)
plt.plot(x, fit4_dn.eval(x=x)-EF, color='xkcd:turquoise', linestyle='--', linewidth=3)

ax.set_xlim(0.0, 0.5)
ax.set_ylim(-0.75, 0.0)
plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=EF, slice_constant='ky')

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\Figs\lamp_dn_ky=-0.72_fit2.png')  # int range 0.05 

In [ ]:
# Plot DN Data with Fits

EF = 16.8

fig, ax = plotting_functions.plot_2D_mpl(x=x_dn, y=y_dn, data=d_dn, xlabel='kx', ylabel='ky', cmap='Blues', EF=EF)

# Raw Data Fits
plt.plot(x, fit1.eval(x=x)-EF, color='xkcd:purple', linestyle='--', linewidth=3)
plt.plot(x, fit2.eval(x=x)-EF, color='xkcd:purple', linestyle='--', linewidth=3)

# plt.plot(x, fit3.eval(x=x)-EF, color='xkcd:purple', linestyle='--', linewidth=3)
# plt.plot(x, fit4.eval(x=x)-EF, color='xkcd:purple', linestyle='--', linewidth=3)

# CNN Data Fits
plt.plot(x, fit1_dn.eval(x=x)-EF, color='xkcd:turquoise', linestyle='--', linewidth=3)
plt.plot(x, fit2_dn.eval(x=x)-EF, color='xkcd:turquoise', linestyle='--', linewidth=3)

# plt.plot(x, fit3_dn.eval(x=x)-EF, color='xkcd:turquoise', linestyle='--', linewidth=3)
# plt.plot(x, fit4_dn.eval(x=x)-EF, color='xkcd:turquoise', linestyle='--', linewidth=3)

ax.set_xlim(0.0, 0.5)
ax.set_ylim(-0.75, 0.0)
plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=EF, slice_constant='ky')

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\Figs\lamp_dn_ky=-0.72_bothfits.png')

In [ ]:
# Plot Raw Data with Fits

EF = 16.8

fig, ax = plotting_functions.plot_2D_mpl(x=x, y=y, data=normed_data, xlabel='kx', ylabel='E - EF (eV)', title=f'ky={slice_val}', cmap='Blues', EF=EF)

# Raw Data Fits
# plt.plot(x, fit1.eval(x=x)-EF, color='xkcd:purple', linestyle='--', linewidth=3)
# plt.plot(x, fit2.eval(x=x)-EF, color='xkcd:purple', linestyle='--', linewidth=3)

# plt.plot(x, fit3.eval(x=x)-EF, color='xkcd:purple', linestyle='--', linewidth=3)
# plt.plot(x, fit4.eval(x=x)-EF, color='xkcd:purple', linestyle='--', linewidth=3)

# CNN Data Fits
# plt.plot(x, fit1_dn.eval(x=x)-EF, color='xkcd:turquoise', linestyle='--', linewidth=3)
# plt.plot(x, fit2_dn.eval(x=x)-EF, color='xkcd:turquoise', linestyle='--', linewidth=3)

plt.plot(x, fit3_dn.eval(x=x)-EF, color='xkcd:turquoise', linestyle='--', linewidth=3)
plt.plot(x, fit4_dn.eval(x=x)-EF, color='xkcd:turquoise', linestyle='--', linewidth=3)

ax.set_xlim(0.0, 0.5)
ax.set_ylim(-0.75, 0.0)
plotting_functions.format_HS_plot(fig, ax, slice_val=slice_val, EF=EF, slice_constant='ky')

# plt.savefig(r'C:\Users\atully\OneDrive\Physics.UBC\DAE\Figs\lamp_raw_ky=-0.72_w_dn_fit.png')